<a href="https://www.kaggle.com/code/krippanandhini/credit-card-fraud-detection?scriptVersionId=163955930" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('/kaggle/input/fraud-detection/fraudTrain.csv')
test_df = pd.read_csv('/kaggle/input/fraud-detection/fraudTest.csv')
df1 = pd.concat([df,test_df])

In [3]:
df1.shape

(1852394, 23)

In [4]:
df1.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06
mean,5.371934e+05,4.173860e+17,7.006357e+01,4.881326e+04,3.853931e+01,-9.022783e+01,8.864367e+04,1.358674e+09,3.853898e+01,-9.022794e+01,5.210015e-03
std,3.669110e+05,1.309115e+18,1.592540e+02,2.688185e+04,5.071470e+00,1.374789e+01,3.014876e+05,1.819508e+07,5.105604e+00,1.375969e+01,7.199217e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902742e+01,-1.666716e+02,0.000000e+00
25%,2.315490e+05,1.800429e+14,9.640000e+00,2.623700e+04,3.466890e+01,-9.679800e+01,7.410000e+02,1.343017e+09,3.474012e+01,-9.689944e+01,0.000000e+00
50%,4.630980e+05,3.521417e+15,4.745000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.443000e+03,1.357089e+09,3.936890e+01,-8.744069e+01,0.000000e+00
75%,8.335758e+05,4.642255e+15,8.310000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.374581e+09,4.195626e+01,-8.024511e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.992100e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.388534e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [5]:
df1 = df1.drop('Unnamed: 0', axis = 1)

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1852394 entries, 0 to 555718
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  object 
 1   cc_num                 int64  
 2   merchant               object 
 3   category               object 
 4   amt                    float64
 5   first                  object 
 6   last                   object 
 7   gender                 object 
 8   street                 object 
 9   city                   object 
 10  state                  object 
 11  zip                    int64  
 12  lat                    float64
 13  long                   float64
 14  city_pop               int64  
 15  job                    object 
 16  dob                    object 
 17  trans_num              object 
 18  unix_time              int64  
 19  merch_lat              float64
 20  merch_long             float64
 21  is_fraud               int64  
dtypes: float64(5), int64(5),

In [7]:
df1.isna().sum()

trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [8]:
fraud_counts = df1['is_fraud'].value_counts()

fig = px.pie(names=fraud_counts.index, values=fraud_counts.values, title='Fraudulent Transactions Distribution')

fig.show()

In [9]:
le = LabelEncoder()
for column in df1.columns:
    if df1[column].dtype == 'object':
        df1[column] = le.fit_transform(df1[column])

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1852394 entries, 0 to 555718
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  int64  
 1   cc_num                 int64  
 2   merchant               int64  
 3   category               int64  
 4   amt                    float64
 5   first                  int64  
 6   last                   int64  
 7   gender                 int64  
 8   street                 int64  
 9   city                   int64  
 10  state                  int64  
 11  zip                    int64  
 12  lat                    float64
 13  long                   float64
 14  city_pop               int64  
 15  job                    int64  
 16  dob                    int64  
 17  trans_num              int64  
 18  unix_time              int64  
 19  merch_lat              float64
 20  merch_long             float64
 21  is_fraud               int64  
dtypes: float64(5), int64(17)

In [11]:
X_train, X_test = train_test_split(df1,test_size = 0.2, random_state = 42)

In [12]:
isf = IsolationForest()
isf.fit(X_train)

IsolationForest()

In [13]:
X_test['y_pred'] = isf.predict(X_test)

In [14]:
X_test['y_pred'] = X_test['y_pred'].replace({-1: 1, 1: 0})
X_test.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,y_pred
244469,1514964,5359543825610251,285,2,59.91,245,144,1,185,62,...,-111.1439,18182,163,570,870220,1379488419,45.274075,-111.649432,0,0
434906,1702272,5540636818935089,284,12,3.96,203,141,1,338,440,...,-71.1605,76383,216,727,845611,1386265705,43.356278,-71.008959,0,0
354659,349469,2720894374956739,24,2,51.17,28,194,0,349,545,...,-82.8823,16305,374,7,1068756,1339759484,42.372483,-83.508020,0,0
197113,1468390,6011438889172900,531,12,2.06,8,5,0,421,204,...,-91.3336,5161,147,871,1174654,1377816625,33.833389,-91.158293,0,1
468148,460913,60495593109,318,13,6.58,274,109,1,454,194,...,-96.7430,1263321,460,93,1134933,1343231435,32.458643,-96.577001,0,1


In [15]:
y_pred = X_test['y_pred']
is_fraud = X_test['is_fraud']

report = classification_report(is_fraud, y_pred)

print(report)

              precision    recall  f1-score   support

           0       1.00      0.71      0.83    368526
           1       0.02      0.94      0.03      1953

    accuracy                           0.71    370479
   macro avg       0.51      0.83      0.43    370479
weighted avg       0.99      0.71      0.83    370479

